<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/gn/MOM_April_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/updated_mom/MinutesOfMeeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install all the dependencies. Place any new installables into requirements.txt file.
!pip install -r /content/drive/MyDrive/requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch

In [4]:
!pip install datasets
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Select an audio file and read it:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
audio_sample = ds[0]["audio"]

# Load the Whisper model with SDPA attention
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en", attn_implementation="sdpa")

# Enable static cache and compile the forward pass
model.generation_config.cache_implementation = "static"
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

# Use the model and processor to transcribe the audio:
input_features = processor(
    audio_sample["array"], sampling_rate=audio_sample["sampling_rate"], return_tensors="pt"
).input_features

# Compile the forward pass
for _ in range(2):
    model.generate(input_features)

# Generate token ids using compiled graph (fast!)
predicted_ids = model.generate(input_features)

# Decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

transcription[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/9.19M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.'

In [5]:
from transformers import AutoProcessor, Wav2Vec2BertForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation", trust_remote_code=True)
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")
model = Wav2Vec2BertForCTC.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")

# audio file is decoded on the fly
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe speech
transcription = processor.batch_decode(predicted_ids)
transcription[0]

inputs["labels"] = processor(text=dataset[0]["text"], return_tensors="pt").input_ids

# compute loss
loss = model(**inputs).loss
round(loss.item(), 2)

librispeech_asr_demo.py:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Generating validation split: 0 examples [00:00, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/299 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

17.04

In [6]:
import torch
from transformers import pipeline

whisper = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", torch_dtype=torch.float16, device="cuda:0")

# transcription = whisper("/content/drive/MyDrive/ColabNotebooks/EarningsCall.wav", return_timestamps=True)

# print(transcription["text"])

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
!pip install deepspeech

ERROR: Could not find a version that satisfies the requirement deepspeech (from versions: none)
ERROR: No matching distribution found for deepspeech


In [24]:
# whisper-large-v3
def transcribe_audio_with_whisper(file):
    whisper = pipeline(
        "automatic-speech-recognition",
        "openai/whisper-large-v3",
        torch_dtype=torch.float16,
        device="cuda:0"
    )

    start_trans = time.time()
    transcription = whisper(file, return_timestamps=True)
    end_trans = time.time()
    full_transcription = transcription["text"]
    trans_time = f"Transcription time: {end_trans - start_trans:.2f} seconds"

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    start_summ = time.time()
    summary = summarizer(full_transcription, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
    end_summ = time.time()
    summ_time = f"Summarization time: {end_summ - start_summ:.2f} seconds"

    return full_transcription, summary, trans_time, summ_time

In [25]:
# Wav2vec2
from transformers import pipeline
import torch
import time

def transcribe_audio_with_wav2vec2(file):
    # Initialize the Wav2Vec2 ASR model
    wav2vec2 = pipeline(
        "automatic-speech-recognition",
        model="facebook/wav2vec2-large-960h",
        torch_dtype=torch.float16,
        device="cuda:0"
    )

    start_trans = time.time()
    # Transcribe the audio with timestamps
    transcription = wav2vec2(file, return_timestamps=True)
    end_trans = time.time()
    full_transcription = transcription["text"]
    trans_time = f"Transcription time: {end_trans - start_trans:.2f} seconds"

    # Initialize the summarization model
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    start_summ = time.time()
    # Summarize the transcription
    summary = summarizer(full_transcription, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
    end_summ = time.time()
    summ_time = f"Summarization time: {end_summ - start_summ:.2f} seconds"

    return full_transcription, summary, trans_time, summ_time


In [26]:
# Dispatcher
def process_based_on_selection(file, choice):
    if choice == "Whisper-large-v3":
        return transcribe_audio_with_whisper(file)
    elif choice == "Facebook - Wav2vec2":
        result = transcribe_audio_with_wav2vec2(file)
    else:
        return "Invalid model", "Invalid model", "", ""

In [27]:
def extract_action_items(full_transcription, keywords):
    # Use default keywords if none provided
    if not keywords:
        keywords = [
            "must", "should", "need to", "investing", "expanding",
            "diversifying", "enhancing", "revenue", "risk", "loss"
        ]
    else:
        keywords = [kw.strip().lower() for kw in keywords.split(",")]

    # Split transcription into sentences
    sentences = re.split(r'(?<=[.!?])\s+', full_transcription)

    # Filter sentences containing any keyword
    action_items = [
        sentence for sentence in sentences
        if any(keyword in sentence.lower() for keyword in keywords)
    ]

    return action_items


In [28]:
def process_action_items(file, choice, keywords):
    transcription, _, _, _ = process_based_on_selection(file, choice)

    start_action = time.time()
    action_items = extract_action_items(transcription, keywords)
    end_action = time.time()
    action_time = f"Action item extraction time: {end_action - start_action:.2f} seconds"

    if action_items:
        formatted_actions = "\n".join([f"{i+1}. {item}" for i, item in enumerate(action_items)])
    else:
        formatted_actions = "No action items found."

    return f"{formatted_actions}\n\n{action_time}"


In [29]:
import nltk
import shutil
import os

# Clean corrupted tokenizer
nltk_data_path = "/usr/local/share/nltk_data/tokenizers/punkt"
if os.path.exists(nltk_data_path):
    shutil.rmtree(nltk_data_path)

# Redownload
nltk.download('punkt', force=True)
nltk.data.path.append("/usr/local/share/nltk_data")  # Ensure path is visible


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [30]:

def evaluate_summary(transcription, summary):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from rouge_score import rouge_scorer
    from nltk.translate.bleu_score import sentence_bleu
    from nltk.translate.meteor_score import meteor_score
    import nltk
    nltk.download('punkt')
    nltk.download('wordnet')

    trans_tokens = transcription.split()
    summ_tokens = summary.split()

    common_tokens = set(trans_tokens).union(set(summ_tokens))
    trans_vector = [1 if token in trans_tokens else 0 for token in common_tokens]
    summ_vector = [1 if token in summ_tokens else 0 for token in common_tokens]

    accuracy = accuracy_score(trans_vector, summ_vector)
    precision = precision_score(trans_vector, summ_vector, zero_division=0)
    recall = recall_score(trans_vector, summ_vector, zero_division=0)
    f1 = f1_score(trans_vector, summ_vector, zero_division=0)

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(transcription, summary)
    rouge = rouge_scores['rougeL'].fmeasure

    # BLEU & METEOR
    bleu = sentence_bleu([trans_tokens], summ_tokens)
    meteor = meteor_score([transcription.split()], summary.split())


    return {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROUGE': rouge,
        'BLEU': bleu,
        'METEOR': meteor
    }


In [31]:
!pip install fpdf

In [33]:
#!pip install fpdf

In [35]:

def process_and_format_outputs(file, choice, keywords):
    import time
    from fpdf import FPDF
    import base64
    from io import BytesIO

    print("=== PROCESS STARTED ===")

    # Transcription & Summary
    transcription, summary, trans_time, summ_time = process_based_on_selection(file, choice)
    print("Transcription and summary complete")

    trans_with_time = f"{transcription}\n\n{trans_time}"
    summ_with_time = f"{summary}\n\n{summ_time}"
    both_times = f"{trans_time}\n{summ_time}"

    # Action Items
    start_action = time.time()
    action_items = extract_action_items(transcription, keywords)
    end_action = time.time()
    action_time = f"Action item extraction time: {end_action - start_action:.2f} seconds"
    print("Action item extraction complete")

    if action_items:
        formatted_actions = "\n".join([f"{i+1}. {item}" for i, item in enumerate(action_items)])
    else:
        formatted_actions = "No action items found."
    actions_with_time = f"{formatted_actions}\n\n"

    # Evaluation + Plot
    try:
        print("Running evaluate_summary()...")
        evaluation_metrics = evaluate_summary(transcription, summary)
        evaluation_results = "\n".join([f"{k}: {v:.2f}" for k, v in evaluation_metrics.items()])
        print("Evaluation successful")
    except Exception as e:
        evaluation_results = f"Error during evaluation: {str(e)}"
        evaluation_metrics = {}
        print("Evaluation failed:", str(e))

    # PDF export
    try:
        pdf_filename = "mom_report.pdf"
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        pdf.multi_cell(0, 10, transcription)
        pdf.multi_cell(0, 10, summary)
        pdf.multi_cell(0, 10, formatted_actions)
        pdf.multi_cell(0, 10, evaluation_results)

        pdf.output(pdf_filename)
        export_path = pdf_filename
        print(f"PDF report saved to {export_path}")
    except Exception as e:
        export_path = None
        print("PDF generation failed:", str(e))

    print("=== PROCESS COMPLETE ===")
    return (
        trans_with_time,
        summ_with_time,
        both_times,
        actions_with_time,
        evaluation_results,
        evaluation_metrics,
        export_path
    )


In [36]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
!pip install gradio
import gradio as gr
from transformers import pipeline
import torch
import time
import re

In [38]:
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


In [39]:
import nltk
import shutil
import os

# Clean corrupted tokenizer
nltk_data_path = "/usr/local/share/nltk_data/tokenizers/punkt"
if os.path.exists(nltk_data_path):
    shutil.rmtree(nltk_data_path)

# Redownload
nltk.download('punkt', force=True)
nltk.data.path.append("/usr/local/share/nltk_data")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [40]:

!pip install gradio
import gradio as gr
from transformers import pipeline
import torch
import time
import re
import plotly.graph_objects as go
from gradio.themes.soft import Soft
from gradio.themes.utils import colors

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    nltk.download('wordnet')

# Custom Gradio theme
custom_theme = Soft(primary_hue=colors.blue, font=["Inter", "sans-serif"])

def generate_summary_plot():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[1, 2, 3], y=[10, 15, 13], mode='lines+markers'))
    fig.update_layout(title="Summary Length Over Time")
    return fig

def show_graphs(eval_metrics):
    fig1 = go.Figure([go.Bar(x=list(eval_metrics.keys()), y=list(eval_metrics.values()))])
    fig1.update_layout(title="Evaluation Metrics - Bar Chart")

    fig2 = go.Figure([go.Pie(labels=list(eval_metrics.keys()), values=list(eval_metrics.values()))])
    fig2.update_layout(title="Evaluation Metrics - Pie Chart")

    fig3 = go.Figure([go.Scatter(x=list(eval_metrics.keys()), y=list(eval_metrics.values()), mode="lines+markers")])
    fig3.update_layout(title="Evaluation Metrics - Line Chart")

    metrics = list(eval_metrics.keys())
    values = list(eval_metrics.values())
    values += values[:1]
    fig4 = go.Figure()
    fig4.add_trace(go.Scatterpolar(r=values, theta=metrics + [metrics[0]], fill='toself'))
    fig4.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])), title="Evaluation Metrics - Radar")

    return (
        gr.update(value=fig1, visible=True),
        gr.update(value=fig2, visible=True),
        gr.update(value=fig3, visible=True),
        gr.update(value=fig4, visible=True)
    )

with gr.Blocks(theme=custom_theme) as demo:
    gr.Markdown("### Minutes of Meeting App")

    evaluation_state = gr.State()

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="Upload your audio file")

            with gr.Accordion("Advanced Settings", open=False):
                dropdown = gr.Dropdown(
                    choices=["Whisper-large-v3", "Facebook - Wav2vec2", "Whisper-Small", "Mozzilla - Deepspeech", "Google - Speech Recognition"],
                    label="Select Model"
                )

                keyword_input = gr.Textbox(
                    label="Custom Action Item Keywords (comma-separated)",
                    placeholder="e.g. must, should, expanding, revenue"
                )

            btn_action = gr.Button("Extract Custom Action Items")
            btn_all = gr.Button("Process All")

            # plot_output = gr.Plot(label="Summary Graph")
            # btn_graph = gr.Button("Show Summary Graph")

            with gr.Accordion("Evaluation Metric Graphs", open=False):
                with gr.Tabs():
                    with gr.TabItem("Bar Chart"):
                        graph1 = gr.Plot(visible=False)
                    with gr.TabItem("Pie Chart"):
                        graph2 = gr.Plot(visible=False)
                    with gr.TabItem("Line Chart"):
                        graph3 = gr.Plot(visible=False)
                    with gr.TabItem("Radar Chart"):
                        graph4 = gr.Plot(visible=False)

            btn_show_graphs = gr.Button("Show Evaluation Metrics")
            btn_show_graphs.click(
                fn=show_graphs,
                inputs=[evaluation_state],
                outputs=[graph1, graph2, graph3, graph4]
            )

            output_plot = gr.HTML(label="Evaluation Plot (Static)")
            output_download = gr.File(label="Download Report")

        with gr.Column(scale=1):
            output_trans = gr.Textbox(label="Transcribed Text", lines=10)
            output_summ = gr.Textbox(label="Summary", lines=6)
            output_time = gr.Textbox(label="Processing Time", lines=2)
            output_action = gr.Textbox(label="Action Items", lines=6)
            output_metrics = gr.Textbox(label="Evaluation Metrics", lines=6)

    btn_action.click(
        fn=process_action_items,
        inputs=[file_input, dropdown, keyword_input],
        outputs=output_action
    )

    btn_all.click(
        fn=process_and_format_outputs,
        inputs=[file_input, dropdown, keyword_input],
        outputs=[
            output_trans,
            output_summ,
            output_time,
            output_action,
            output_metrics,
            evaluation_state,
            output_download
        ]
    )

    #btn_graph.click(fn=generate_summary_plot, outputs=plot_output)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bbad19d5c66670dc70.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
